<a href="https://colab.research.google.com/github/iresh-rajitha/Character-seperate-using-CV/blob/main/Selenium_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt-get install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
!pip install selenium-wire

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,498 kB]
Get:14 https://developer.dow

In [ ]:
# prompt: get a file from google drive

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# get downloaded_fonts.txt file from google drive if not exist
import os
if not os.path.exists("downloaded_fonts.txt"):
  !cp /content/drive/MyDrive/sinhala_fonts/downloaded_fonts.txt .

In [ ]:
# prompt: read the each line of the font_names.txt file and make a font name array and remove the names which in downloaded_fonts.txt

# Read font names from font_names.txt
with open('font_names.txt', 'r') as f:
    font_names = [line.strip() for line in f]

# Read downloaded font names from downloaded_fonts.txt
with open('downloaded_fonts.txt', 'r') as f:
    downloaded_fonts = [line.strip() for line in f]

# Remove downloaded fonts from the font_names list
font_names = [font for font in font_names if font not in downloaded_fonts]

# Print the resulting font name array
print(font_names)


In [ ]:
# Create the downloads folder if it doesn't exist
if not os.path.exists("downloads"):
    os.makedirs("downloads")

In [ ]:
# Step 3: Set up the Selenium WebDriver with Chrome options
from seleniumwire import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Set up Chrome options
chrome_options = Options()
# chrome_options.add_argument("--incognito")  # Open Chrome in incognito mode
chrome_options.add_argument("--headless")  # Ensure the browser runs in headless mode
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
# chrome_options.binary_location = "/usr/bin/chromium-browser"

# Set the path to your WebDriver executable
driver_path = "/usr/bin/chromedriver"
download_link = ""
# Initialize the WebDriver
# service = Service(driver_path)
def get_font_link(font):
  driver = webdriver.Chrome(options=chrome_options)
  try:
      # Open a website
      driver.get(f'https://sinhala-fonts.org/fonts/{font}')

      # Wait until the page is loaded
      WebDriverWait(driver, 12).until(
          EC.presence_of_element_located((By.TAG_NAME, "body"))
      )

      # Execute JavaScript to trigger the download
      java_script = """
          document.getElementById("id_method").value = "app";
          document.getElementById("fontdownloadsubmit").click();
          """
      driver.execute_script(java_script)

      # Wait for the href of the download link to change
      WebDriverWait(driver, 30).until(
          EC.text_to_be_present_in_element_attribute(
              (By.XPATH, "//a[starts-with(@href, '/downloadfiles/')]"),
              "href",
              "/downloadfiles/"
          )
      )

      # Get all a tags href
      a_tags = driver.find_elements(By.XPATH, "//a")
      for a_tag in a_tags:
          # Filter link start with https://sinhala-fonts.org/downloadfiles/
          if a_tag.get_attribute("href").startswith("https://sinhala-fonts.org/downloadfiles/"):
              print(a_tag.get_attribute("href"))
              download_link = a_tag.get_attribute("href")

      # Clear cookies and storage
      driver.delete_all_cookies()  # Clears all cookies
      driver.execute_script("window.localStorage.clear();")  # Clears local storage
      driver.execute_script("window.sessionStorage.clear();")  # Clears session storage
  finally:
      # Close the browser
      driver.quit()
  return download_link

In [ ]:
from tqdm.notebook import tqdm
import requests
# Initialize a counter for downloaded fonts
downloaded_count = 0

# Create an empty list to store downloaded font names
downloaded_fonts = []

# Loop through the font names
for font in tqdm(font_names, desc='Processing fonts'):
    try:
        # Call the function to get the download link
        download_link = get_font_link(font)
        response = requests.get(download_link)

        if response.headers.get('Content-Type') == 'application/zip':
          with open(f"downloads/{font}.zip", "wb") as f:
            f.write(response.content)
        else:
          raise Exception("Not a zip file")

        # write to downloaded_fonts.txt as append new line
        with open("downloaded_fonts.txt", "a") as file:
            file.write(font + "\n")

        # Add the font name to the list of downloaded fonts
        downloaded_fonts.append(font)

        # Increment the downloaded count
        downloaded_count += 1

        # Print the progress
        print(f"Downloaded {downloaded_count} fonts out of {len(font_names)}")

    except Exception as e:
        print(f"Error downloading font {font}: {e}")
        break

    finally:
        !cp -r /content/downloads /content/drive/MyDrive/sinhala_fonts
        with open('/content/drive/MyDrive/sinhala_fonts/downloaded_fonts.txt', 'w') as f:
            for font in downloaded_fonts:
                f.write(font + '\n')